In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

### Биномиальное распределение и MDE

In [2]:
adsmart_data = pd.read_csv("../data/ad_smart.csv")
adsmart_data

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
...,...,...,...,...,...,...,...,...,...
8072,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0,0
8073,ffea3210-2c3e-426f-a77d-0aa72e73b20f,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
8074,ffeaa0f1-1d72-4ba9-afb4-314b3b00a7c7,control,2020-07-04,9,Generic Smartphone,6,Chrome Mobile,0,0
8075,ffeeed62-3f7c-4a6e-8ba7-95d303d40969,exposed,2020-07-05,15,Samsung SM-A515F,6,Samsung Internet,0,0


Сайт внедряет новую умную рекламу — AdSmart.<br>Задача: проверить, повлияет ли он на выбор ответов "да" в некоем опроснике.

$H_0:$ Нет существенной разницы между показателями успешности рекламы в группах <br>
$H_1:$ Существует значительная разница между показателями успешности рекламы в группах

Для начала очистим данные: оставьте только кейсы, в которых пользователи ответили на вопрос (есть значение "да" или "нет") и оставьте только колонку "да"

In [3]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
adsmart_data = adsmart_data[(adsmart_data.yes != 0) | (adsmart_data.no != 0)].drop(columns=['no']).reset_index(drop=True)
adsmart_data

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0
1,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1
2,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,6,Chrome Mobile,0
3,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1
4,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0
...,...,...,...,...,...,...,...,...
1238,ffa08ff9-a132-4051-aef5-01a9c79367bc,exposed,2020-07-05,21,Generic Smartphone,6,Chrome Mobile,1
1239,ffb176df-ecd2-45d3-b05f-05b173a093a7,exposed,2020-07-04,1,Generic Smartphone,6,Chrome Mobile,1
1240,ffb79718-6f25-4896-b6b3-e58b80a6e147,control,2020-07-09,7,Generic Smartphone,6,Chrome Mobile,0
1241,ffca1153-c182-4f32-9e90-2a6008417497,control,2020-07-10,16,Generic Smartphone,6,Chrome Mobile,0


Используя группу `control` в полученном датасете, найдите среднее количество (`baseline_rate`) ответов "да". Можно округлить до двух знаков после точки.

In [4]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
A = adsmart_data[adsmart_data.experiment == 'control']
B = adsmart_data[adsmart_data.experiment == 'exposed']
baseline_rate = round(A.yes.mean(), 2)
baseline_rate

0.45

Хотим использовать тест пропорций. Рассчитайте effect size для заданного MDE при соотношении выборок 50/50

In [5]:
from statsmodels.stats.api import proportion_effectsize
MDE = 0.05
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
effect_size = proportion_effectsize(baseline_rate, baseline_rate + MDE)
effect_size

-0.10016742116155997

И необходимое количество наблюдений для контрольной и тестовой выборок (округлить вверх)

In [7]:
from statsmodels.stats.power import zt_ind_solve_power

power = 0.8
alpha = 0.05
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
np.ceil(zt_ind_solve_power(effect_size = effect_size, alpha = alpha, power = power, alternative='two-sided'))

1565.0

Достаточно ли наблюдений для эксперимента с заданным MDE?

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

### A/A-тестирование

In [5]:
sales_data = pd.read_csv("../data/store_sales_test.csv")
cleaning_data = sales_data[(sales_data.family == 'CLEANING')]
cleaning_data

,id,date,store_nbr,family,onpromotion
7,3000895,2017-08-16,1,CLEANING,25
40,3000928,2017-08-16,10,CLEANING,30
73,3000961,2017-08-16,11,CLEANING,35
106,3000994,2017-08-16,12,CLEANING,29
139,3001027,2017-08-16,13,CLEANING,29
...,...,...,...,...,...
28354,3029242,2017-08-31,54,CLEANING,16
28387,3029275,2017-08-31,6,CLEANING,15
28420,3029308,2017-08-31,7,CLEANING,10
28453,3029341,2017-08-31,8,CLEANING,8


Проведя A/A-тесты, выделите 2 гомогенные группы по 15 различных `store_nbr` таких, что:
* В течение всего периода подтверждается $H_0$ t-теста
* Распределение p-values не слишком далеко от равномерного по Колмогорову-Смирнову
* Выборки однородны ГС

In [15]:
from scipy.stats import ttest_ind, kstest

alpha = 0.05
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
for i in range(40):
    r = np.random.choice(cleaning_data.store_nbr.unique(), 30, replace=False)
    a1_i = r[:10]
    a2_i = r[10:]
    a1 = cleaning_data[cleaning_data.store_nbr.isin(a1_i)]
    a2 = cleaning_data[cleaning_data.store_nbr.isin(a2_i)]
    pvals = []
    for date in a1.date.unique():
        a1_vals = a1[a1.date == date].onpromotion
        a2_vals = a2[a2.date == date].onpromotion
        pvals.append(ttest_ind(a1_vals, a2_vals, equal_var=False).pvalue)
    
    if (kstest(pvals, "uniform").pvalue > alpha) and (min(pvals) > alpha):
        is_a1_h = kstest(a1_vals, cleaning_data.onpromotion).pvalue > 0.05
        is_a2_h = kstest(a2_vals, cleaning_data.onpromotion).pvalue > 0.05
        if is_a1_h and is_a2_h:
            print(a1_i)
            print(a2_i)
            break

[48 26 16 18 42  1 21 22 41 31]
[ 4 43 24 53 28 27 54 13 17 25 20 36 11 15 38  6 19  9 32  8]


### A/A/B-тестирование
На практике не всегда есть время/средства для предварительного A/A-тестирования. В таких случаях может использоваться A/A/B-тест: данные делятся на три группы, две контрольные и одну тестовую. Мы одновременно проводим A/B-эксперимент и проверяем систему на A/A-части:

In [27]:
logs_data = pd.read_csv("../data/logs_exp_us.csv", sep='\t')
logs_data = logs_data.drop_duplicates()
logs_data['ExpId'] = logs_data['ExpId'].replace({246: 'control_1', 247: 'control_2', 248: 'test'})
logs_data = logs_data.rename(columns={'ExpId': 'TestGroup'})
logs_data

,EventName,DeviceIDHash,EventTimestamp,TestGroup
0,MainScreenAppear,4575588528974610257,1564029816,control_1
1,MainScreenAppear,7416695313311560658,1564053102,control_1
2,PaymentScreenSuccessful,3518123091307005509,1564054127,test
3,CartScreenAppear,3518123091307005509,1564054127,test
4,PaymentScreenSuccessful,6217807653094995999,1564055322,test
...,...,...,...,...
244121,MainScreenAppear,4599628364049201812,1565212345,control_2
244122,MainScreenAppear,5849806612437486590,1565212439,control_1
244123,MainScreenAppear,5746969938801999050,1565212483,control_1
244124,MainScreenAppear,5746969938801999050,1565212498,control_1


In [29]:
logs_data[logs_data.EventName == 'MainScreenAppear'].TestGroup.value_counts()

TestGroup
test         41175
control_2    39677
control_1    38249
Name: count, dtype: int64

Проверяем, будут ли пользователи тестовой группы чаще переходить к покупке.

Будем использовать z-тест пропорций: за количество успехов считаем количество событий `PaymentScreenSuccessful`, количество всех экспериментов — события `MainScreenAppear`.

Для начала проверьте гипотезу о нестатзначимости разности средних для двух контрольных групп (A/A)

In [23]:
from statsmodels.stats.proportion import proportions_ztest

alpha = 0.05

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
control_1 = logs_data[logs_data.TestGroup == 'control_1']
control_2 = logs_data[logs_data.TestGroup == 'control_2']

count_1 = control_1[control_1.EventName == 'PaymentScreenSuccessful'].shape[0]
count_2 = control_2[control_2.EventName == 'PaymentScreenSuccessful'].shape[0]
nobs_1 = control_1[control_1.EventName == 'MainScreenAppear'].shape[0]
nobs_2 = control_2[control_2.EventName == 'MainScreenAppear'].shape[0]
stat, pval_controls = proportions_ztest(count=np.array([count_1, count_2]),
                                        nobs=np.array([nobs_1, nobs_2]))
pval_controls, pval_controls < alpha

(2.1148551469454695e-73, True)

Как и в случае обычного A/A-теста необходимо продолжительное наблюдение за результатами. Если мы будем стабильно получать статзначимость разницы между контрольными группами — это повод перепроверить систему тестирования, вмешательство в эксперимент неучтённых факторов и разбиение на группы.

Теперь объедините две контрольных группы в одну и проверьте гипотезу для обычного A/B-теста

In [24]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
test = logs_data[logs_data.TestGroup == 'test']

control_count = count_1 + count_2
control_nobs = nobs_1 + nobs_2
test_count = test[test.EventName == 'PaymentScreenSuccessful'].shape[0]
test_nobs = test[test.EventName == 'MainScreenAppear'].shape[0]
stat, pval = proportions_ztest(count=np.array([control_count, test_count]),
                               nobs=np.array([control_nobs, test_nobs]))
pval, pval < alpha

(5.397051376310148e-07, True)

Какой вывод можно сделать?

И, наконец, сравните три группы попарно. Используйте подход A/B/C-тестов с поправкой Бонферрони. Изменится ли вывод?

In [25]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
stat, pval_c1_test = proportions_ztest(count=np.array([count_1, test_count]),
                                       nobs=np.array([nobs_1, test_nobs]))
stat, pval_c2_test = proportions_ztest(count=np.array([count_2, test_count]),
                                       nobs=np.array([nobs_2, test_nobs]))

pval_c1_test, pval_c2_test

(1.0451245019488738e-06, 1.0713756684652275e-41)

In [26]:
bonferroni_alpha = alpha / 3
pval_controls < bonferroni_alpha, pval_c1_test < bonferroni_alpha, pval_c2_test < bonferroni_alpha

(True, True, True)

В жизни может использоваться деление по аналогии и на большее кол-во частей, например, A/A/B/B-тест и другие.